In [1]:
import pandas as pd
import numpy as np
import geojson as gj

In [2]:
LAPD_file = "../static/data/LAPD_Divisions.geojson"
LAPD_df = pd.read_json(LAPD_file).T # THIS IS NOT WORKING

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [ ]:
# Load json to dataframe
crime_year = "crime_2013"
file_to_load = f'../static/data/{crime_year}.json'
crime_df = pd.read_json(file_to_load).T

In [ ]:
# # Numberize the Lat and Lon columns
# crime_df.reset_index(drop = True, inplace = True)
# crime_df["LAT"] = pd.to_numeric(crime_df["LAT"])
# crime_df["LON"] = pd.to_numeric(crime_df["LON"])

# crime_df.head()

In [ ]:
# # Sanity Checks
# crime_df.head()
# crime_df.groupby('Year').LON.count()
# crime_df.groupby('Year').LON.value_counts()

# Aggregate by Lat and Lon
grouped_df = crime_df.groupby(['LAPD_Area']).agg({'Crime_Type':['count']}) 
grouped_df.columns = ['Crime_Counts'] 
grouped_df.reset_index(level=['LAPD_Area'],inplace=True) 
grouped_df.head()

In [ ]:
# Sanity check data types
grouped_df.dtypes

In [ ]:
# Create DF for LAPD_Divisions_geojson

In [ ]:
# Join grouped_df (crime) with LAPD_divsions - join by area

In [ ]:
def data2geojson(df, filename):
    features = []
    insert_features = lambda X: features.append(
            gj.Feature(geometry=gj.Point((X["LON"],
                                         X["LAT"]
                                         )),
                            properties=dict(crime_counts=X["Crime_Counts"])))
    df.apply(insert_features, axis=1)
    with open(filename, 'w', encoding='utf8') as fp:
        gj.dump(gj.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

In [ ]:
outputFilePath = '../static/data/'
dispFileName = f'{outputFilePath}{crime_year}.geojson'

data2geojson(grouped_df, dispFileName)